## NYISO Load Prediction
- Objective: Utilize the NBEATS model to predict NYISO load data for 2023-12-31 using historical data from 2013-01-01 to 2023-12-30.
- Zones: `N.Y.C.`, `NORTH`, `CENTRL`
- Scaling methods: [definition](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html)
     - [`identity`](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html#std-statistics)
     - [`standard`](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html#std-statistics)
     - [`minmax`](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html#minmax-statistics)
     - [`robust`](https://nixtlaverse.nixtla.io/neuralforecast/common.scalers.html#robust-statistics)
     - `revin`:  learnable normalization parameters are added on top of the usual normalization technique.

In [1]:
import warnings
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display, clear_output

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Select, CustomJS, DatetimeTickFormatter
from bokeh.layouts import column
from bokeh.palettes import Category10

from ipywidgets import widgets, HBox, VBox
from ipywidgets.embed import embed_minimal_html


# Suppress warnings
warnings.filterwarnings("ignore")
# Load filtered data from pickle files
with open('train_dfs.pkl', 'rb') as f:
    train_dfs = pickle.load(f)

with open('test_dfs.pkl', 'rb') as f:
    test_dfs = pickle.load(f)
    
with open('all_prediction_dfs.pkl', 'rb') as f:
    all_prediction_dfs = pickle.load(f)

with open('zones.pkl', 'rb') as f:
    zones = pickle.load(f)

filtered_data_reconstructed = {zone: pd.concat([train_df, test_dfs[zone]]) for zone, train_df in train_dfs.items()}

### PCA + NBEATS Method
1. Build Eigenspace: Use the first 10 principal components (PCs) from the training data to capture the main patterns.
2. Reconstruct Training Data: Rebuild the training data using these principal components.
3. Calculate Residuals: Measure the difference between the original and reconstructed training data to obtain residuals.
4. Apply to Test Data: Use the eigenspace from the training data to reconstruct and analyze the test data.

In [2]:
def plot_reconstructed_ts(df, zone):
    # # Plot the residuals for weekdays and weekends separately
    df = df.sort_index()
    plt.figure(figsize=(14, 10))
    
    plt.subplot(3, 1, 1)
    plt.plot(df.index, df['y'], label=f'Original {zone} Data')
    plt.title('Original Data')
    plt.xlabel('Timestamp')
    plt.ylabel('Value')
    plt.legend()
    
    plt.subplot(3, 1, 2)
    plt.plot(df.index, df['reconstructed'], label='Reconstructed Data using PC1', color='green')
    plt.title('Reconstructed Data by First 10 PCs')
    plt.xlabel('Timestamp')
    plt.ylabel('Value')
    plt.legend()
    
    plt.subplot(3, 1, 3)
    plt.plot(df.index, df['residuals'], label='Residuals', color='blue')
    plt.title('Residuals (Original - Reconstructed Data)')
    plt.xlabel('Timestamp')
    plt.ylabel('Residual Value')
    plt.legend()
    
    plt.tight_layout()
    plt.show()


# Initial zone
initial_zone = zones[0]

# Create output widgets for plots
output_reconstructed = widgets.Output()

# Display initial plots
with output_reconstructed:
    clear_output(wait=True)
    fig_reconstructed = plot_reconstructed_ts(filtered_data_reconstructed[initial_zone], initial_zone)
    display(fig_reconstructed)

# Define update function for dropdown
def update_plot(change):
    selected_zone = change['new']
    
    # Update reconstructed plot
    with output_reconstructed:
        clear_output(wait=True)
        fig_reconstructed = plot_reconstructed_ts(filtered_data_reconstructed[selected_zone], selected_zone)
        display(fig_reconstructed)


# Create dropdown menu
dropdown = widgets.Dropdown(
    options=zones,
    value=initial_zone,
    description='Select Zone:',
    style={'description_width': 'initial'}
)

# Attach update function to dropdown
dropdown.observe(update_plot, names='value')

# Arrange the dropdown and plots in a layout with two columns
layout = VBox([dropdown, HBox([output_reconstructed])])

# Display the layout
display(layout)

embed_minimal_html('export.html', views=[dropdown, output_reconstructed], title='Widgets Export')

In [3]:
# Ensure Bokeh outputs to the notebook
output_notebook()

def plot_prediction(plot_df, zone):
    if "unique_id" in plot_df.columns:
        plot_df.drop("unique_id", axis=1, inplace=True)
        
    source = ColumnDataSource(plot_df)
    
    p = figure(title=f"NYISO - {zone}", x_axis_type='datetime', x_axis_label='Date', y_axis_label='Load', width=1400, height=850)
    
    colors = Category10[len(plot_df.columns)]
    
    for i, col in enumerate(plot_df.columns):
        if col != 'ds':  # Exclude the index column from plotting
            p.line(x='ds', y=col, source=source, line_width=2, color=colors[i], legend_label=col)
    
    p.legend.title = ''
    p.legend.title_text_font_size = '12pt'
    p.legend.label_text_font_size = '10pt'
    p.legend.location = 'bottom_left'
    
    p.xaxis.formatter = DatetimeTickFormatter(
        days="%Y-%m-%d",
        months="%Y-%m-%d",
        years="%Y-%m-%d"
    )
    
    return p


# Create output widgets for plots
output_prediction = widgets.Output()

# Initial zone
initial_zone = zones[0]
fig_prediction = plot_prediction(all_prediction_dfs[initial_zone].iloc[-168:], initial_zone)

# Show the initial plot
with output_prediction:
    show(fig_prediction, notebook_handle=True)

# Define update function for dropdown
def update_plot(change):
    selected_zone = change['new']
    
    # Update prediction plot
    with output_prediction:
        clear_output(wait=True)
        fig_prediction = plot_prediction(all_prediction_dfs[selected_zone].iloc[-168:], selected_zone)
        show(fig_prediction, notebook_handle=True)

# Create dropdown menu
dropdown = widgets.Dropdown(
    options=zones,
    value=initial_zone,
    description='Select Zone:',
    style={'description_width': 'initial'}
)

# Attach update function to dropdown
dropdown.observe(update_plot, names='value')

# Arrange the dropdown and plots in a layout
layout = widgets.VBox([dropdown, output_prediction])

# Display the layout
display(layout)
# Display the initial plot only after layout setup
initial_zone = dropdown.value
with output_prediction:
    fig_prediction = plot_prediction(all_prediction_dfs[initial_zone].iloc[-168:], initial_zone)
    show(fig_prediction, notebook_handle=True)

# Optionally, save the notebook with the widgets embedded
embed_minimal_html('export_prediction.html', views=[dropdown, output_prediction], title='Widgets Prediction Export')


Loading BokehJS ...